In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

## MLP モデルのPyTorchによる実装
基礎編でnumpyを用いて実装したMLPモデル、誤差逆伝搬等をPyTorchで書いてみます。

### PyTorch Tensor
基礎編numpy版ではデータをnumpy array型にして扱っていました。
numpy arrayはnumpyはもちろんmatplotlibなど非常に多くのPythonライブラリで使用できるのですが、残念ながらPyTorchモデルの入力には使用できません。
PyTorchモデルへの入力はTorch Tensor型に変換する必要があります。

PyTorch Tensorは`torch.tensor(3.14)`のようにして定義することができます。

In [ ]:
import torch
a = torch.tensor(3.14)
b = torch.tensor(3)
c = torch.tensor([1, 2, 3])
print(a)
print(b)
print(c)

Numpy arrayからは、同様に`torch.tensor`を使用するか、`from_numpy`を使うことでtensor型に変換できます。

In [ ]:
import numpy as np
x_numpy = np.array([1, 2, 3])
x_tensor_1 = torch.tensor(x_numpy)
x_tensor_2 = torch.from_numpy(x_numpy)
print(x_numpy)
print(x_tensor_1)
print(x_tensor_2)

逆にtorch tensorからnumpy arrayに戻すときは`.numpy()`でできます。

In [ ]:
x_tensor_1.numpy()

tensor型同士の演算はnumpyと同様に行うことができます。

In [ ]:
a = torch.tensor([1, 2])
b = torch.tensor([3, 4])
print("a          = ", a)
print("b          = ", b)
print("2 * a      = ", 2 * a)
print("a + b      = ", a + b)
print("a - b      = ", a - b)
print("a * b      = ", a * b)
print("square(a)  = ", torch.square(a))
print("sigmoid(a) = ", torch.sigmoid(a))

#### PyTorch Tensor型と自動微分(autograd)

PyTorch tensorは自動微分の機能が備わっています。この機能により、複雑な深層学習モデルの学習を容易に実行することが可能となります。

例として、$y=x_1^2 + x_2^2$の$(x_1, x_2)=(4.0, 3.0)$での勾配を求めてみます。
手で計算すると
$$
\begin{align*}
\left. \frac{\partial y}{\partial x_1} \right|_{x_1=4.0} = \left. 2\cdot x_1 \right|_{x_1=4.0} = 8.0 \\
\left. \frac{\partial y}{\partial x_2} \right|_{x_2=3.0} = \left. 2\cdot x_2 \right|_{x_2=3.0} = 6.0
\end{align*}
$$
となります。

自動微分の機能を使用するには`requires_grad=True`を引数に指定してtensorを作成します。

In [ ]:
x1 = torch.tensor(4.0, requires_grad=True)
x2 = torch.tensor(3.0, requires_grad=True)
y = x1 ** 2 + x2 ** 2

ただ値の演算が行われているだけに見えますが、計算グラフが構築されています。
例えば以下のようにして計算グラフを可視化することができます。

In [ ]:
from torchviz import make_dot

make_dot(y, params={"x1": x1, "x2": x2, "y": y})

`.backward()`を実行すると、その変数(ここではy)の計算グラフから自動微分が実行され、計算グラフ内の各tensorに勾配の値が記録されます。

In [ ]:
y.backward()
print("x1の勾配: ", x1.grad)
print("x2の勾配: ", x2.grad)

勾配情報が付加されたtensorを他のライブラリで使う際には注意が必要です。
例えば`.numpy()`でnumpy arrayに変換しようとするとうまくいきません。
```python
x1.numpy()
>>> RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
```

`.detach()`で勾配情報を切り離す必要があります。
```python
x1.detach().numpy()
>>> array(4., dtype=float32)
```

PyTorchでは深層学習モデルのパラメータを勾配計算可能なtensorとして扱い、自動微分可能な形で演算を実装することで、深層学習モデルの効率的な学習を可能にしています。

### モデル定義
基礎編numpy版では
```python
# パーセプトロンのパラメータ
w1 = np.random.randn(2, 3)  # 入力ノード数=2, 出力ノード数=3
b1 = np.random.randn(3)  # 出力ノード数=3
w2 = np.random.randn(3, 1)  # 入力ノード数=3, 出力ノード数=1
b2 = np.random.randn(1)  # 出力ノード数=1

# 順伝搬
a1 = np.dot(x, w1) + b1
z1 = sigmoid(a1)
a2 = np.dot(z1, w2) + b2
y = sigmoid(a2)
```
のようにしてMLPを実装していました。
PyTorchでは
```python
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=1),
    nn.Sigmoid(),
)
model(x)
```
のように書きます。

<details>
<summary>import torch.nn as nn について</summary>

PyTorchのnnモジュールを`nn`という名前でimportしていますが、他にお好みで
```python
import torch
torch.nn.Linear(in_features=2, out_features=3)
```
や
```python
from torch.nn import Linear
Linear(in_features=2, out_features=3)
```
のようにも書けます。
</details>

`Linear` は行列計算($y=Wx+b$)をする関数です。
活性化関数(ここでは`Sigmoid`)と組み合わせることで隠れ層を構成します。
`Linear`の詳細は[公式のドキュメント](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)を参照することでわかります。
ドキュメントを見ると、
```python
torch.nn.Linear(
    in_features,
    out_features,
    bias=True,
    dtype=None
)
```
のような引数を持つことがわかります。また、各引数の意味は、
* `in_features`:	size of each input sample.
* `out_features`:	size of each output sample.
* `bias`:	If set to False, the layer will not learn an additive bias. Default: True.

のようになっています。隠れ層の入出力ノードの数、バイアス項の有無が指定できることがわかります。
知らない関数を使うときは、必ずドキュメントを読んで、関数の入出力、引数、デフォルトの値などを確認するようにしましょう。


PyTorch Model (上の例では`model`)は`torchinfo`等の外部パッケージを使用することで、モデルの構成が確認できます。

In [ ]:
import torchinfo

import torch.nn as nn
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=1),
    nn.Sigmoid(),
)
torchinfo.summary(model, input_size=(1, 2), col_names=["output_size", "num_params"])

Numpyで定義したパラメータ数(w1:6個、b1:3個、w2:3個、b2:1個、計13個)と一致していることを確認してください。

### 学習(誤差逆伝搬)

基礎編numpy版では
```python
# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 2000  # 繰り返し回数
for _ in range(num_steps):
    # 順伝搬させる
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    y = sigmoid(a2)  # パーセプトロンの出力

    # 一次微分を求めるために逆伝搬させる
    grad_a2 = y - t
    grad_w2 = np.einsum("ij,ik->ijk", z1, grad_a2)  # grad_w2 = z1 * grad_a2
    grad_b2 = 1.0 * grad_a2

    grad_a1 = grad_sigmoid(a1) * (grad_a2 * w2.T)
    grad_w1 = np.einsum("ij,ik->ijk", x, grad_a1)  # grad_w1 = x * grad_a1
    grad_b1 = 1.0 * grad_a1

    # パラメータの更新 (mean(axis=0): 全てのデータ点に対しての平均値を使う)
    w2 = w2 - learning_rate * grad_w2.mean(axis=0)
    b2 = b2 - learning_rate * grad_b2.mean(axis=0)
    w1 = w1 - learning_rate * grad_w1.mean(axis=0)
    b1 = b1 - learning_rate * grad_b1.mean(axis=0)
```
のように勾配を計算し、パラメータのアップデートを行っていました。
PyTorchでは
```python
import torch
import torch.nn as nn

# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 2000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()
```
となります。`loss.backward()`を実行することにより、誤差逆伝搬が実行され、各モデルパラメータの誤差が計算されます。
`optimizer.step()`では、optimizer(ここではSGD)の更新式に従ってパラメータの更新が行われます。

```python
import torch
import torch.nn as nn
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
```

### これまでのまとめ

ここまでのコードをまとめて学習を実行してみましょう。

In [ ]:
import torch
import torch.nn as nn

# データ点の取得
x, t = utils.dataset_for_mlp()

# numpy -> torch.tensor
x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=1),
    nn.Sigmoid(),
)

# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 2000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# トレーニング
for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

PyTorch Model (上の例では`model`)は`torchinfo`等の外部パッケージを使用することで、モデルの構成が確認できます。

### モデルの拡張

層の数を増やしてみましょう。新たな層を重ねることで層の数を増やすことができます。

In [ ]:
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=3),
    nn.Sigmoid(),
    nn.Linear(in_features=3, out_features=1),
    nn.Sigmoid(),
)

torchinfo.summary(model, input_size=x.shape, col_names=["output_size", "num_params"])

モデルのパラメータの数が増えていることがわかります。

次に、ノードの数を増やしてみましょう。

In [ ]:
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=128),
    nn.Sigmoid(),
    nn.Linear(in_features=128, out_features=128),
    nn.Sigmoid(),
    nn.Linear(in_features=128, out_features=128),
    nn.Sigmoid(),
    nn.Linear(in_features=128, out_features=128),
    nn.Sigmoid(),
    nn.Linear(in_features=128, out_features=128),
    nn.Sigmoid(),
    nn.Linear(in_features=128, out_features=1),
    nn.Sigmoid(),
)

torchinfo.summary(model, input_size=x.shape, col_names=["output_size", "num_params"])

パラメータの数が大きく増えたことがわかります。
MLPにおいては、パラメータの数は、ノード数の2乗で増加します。

このモデルを使って学習させてみましょう。

In [ ]:
learning_rate = 0.01  # ステップ幅
num_steps = 3000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# トレーニング
for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

これまでは活性化関数としてシグモイド関数(`sigmoid`)を使っていました。昔はsigmoid関数やtanh関数がよく使われていましたが、最近はReLU関数がよく使われます。
$$
  ReLU = \begin{cases}
    x & (x \geq 0) \\
    0 & (x < 0)
  \end{cases}
$$

ReLUが好まれる理由については、別のnotebook(ActivationFunction.ipynb)を参照してください。

ReLUを使って学習がどのようになるか確認してみましょう。

In [ ]:
import torch
import torch.nn as nn

# データ点の取得
x, t = utils.dataset_for_mlp()

# numpy -> torch.tensor
x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
    nn.Sigmoid(),
)

learning_rate = 0.01  # ステップ幅
num_steps = 3000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# トレーニング
for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

深層学習をトレーニングするにあたって、最適化関数(optimizer)も非常に重要な要素です。
確率的勾配降下法(SGD)の他によく使われるアルゴリズムとして adam があります。
adamを使ってみると、どのようになるでしょうか。

In [ ]:
import torch
import torch.nn as nn

# データ点の取得
x, t = utils.dataset_for_mlp()

# numpy -> torch.tensor
x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
    nn.Sigmoid(),
)

num_steps = 3000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# トレーニング
for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

## PyTorch モデルの定義方法
PyTorchモデルを定義する方法はいくつかあります。
最も簡単なのが`Sequential`を使った方法で、これまでの例では全てこの方法でモデルを定義してきました。
一方で、少し複雑なモデルを考えると、`Sequential`モデルで対応できなくなってきます。
一例としてResidual Network(ResNet)で使われるskip connectionを考えてみます。
skip connectionは
$$
y = f_2(x + f_1(x))
$$
のように、入力を２つの経路に分け、片方はMLP、もう片方はそのまま後ろのレイヤーに接続します。
このようなモデルは、途中入出力の分岐があるため、`Sequential`モデルでは実装できません。

PyTorchモデルを定義する方法として、`Module`クラスのサブクラスを作る方法があります。
`Module`クラスをカスタマイズすることができるので、特殊な学習をさせたいときなど、高度な深層学習モデルを扱うときに使われることもあります。
`forward`という関数の中でモデル内のレイヤーの関係を定義します。

In [ ]:
# Modelクラスを継承して新しいクラスを作成します
import torch.nn as nn


class myModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear_1 = nn.Linear(in_features=2, out_features=128)
        self.activation_1 = nn.ReLU()
        self.linear_2 = nn.Linear(in_features=128, out_features=128)
        self.activation_2 = nn.ReLU()
        self.linear_3 = nn.Linear(in_features=128, out_features=128)
        self.activation_3 = nn.ReLU()
        self.linear_4 = nn.Linear(in_features=128, out_features=128)
        self.activation_4 = nn.ReLU()
        self.linear_5 = nn.Linear(in_features=128, out_features=1)
        self.activation_5 = nn.Sigmoid()

    def forward(self, inputs):
        x = self.linear_1(inputs)
        x = self.activation_1(x)
        x = x + self.linear_2(x)  # skip connection
        x = self.activation_2(x)
        x = x + self.linear_3(x)  # skip connection
        x = self.activation_3(x)
        x = x + self.linear_4(x)  # skip connection
        x = self.activation_4(x)
        x = self.linear_5(x)
        x = self.activation_5(x)
        return x


model = myModel()

In [ ]:
import torch
import torch.nn as nn

# データ点の取得
x, t = utils.dataset_for_mlp()

# numpy -> torch.tensor
x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

num_steps = 3000  # 繰り返し回数

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# トレーニング
for i_epoch in range(num_steps):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

## 補足: tqdm
PyTorchは学習中にその進捗を表示する機能を提供していません。
長い学習中に現在の完了率がわからないと困るので、tqdmというライブラリが良く使われます。
tqdmはiterativeオブジェクトラップするだけで機能します。

In [ ]:
import time
from tqdm import tqdm
for _ in tqdm([1, 2, 3, 4, 5]):
    time.sleep(1)  # 1秒スリープ
    pass

学習のためのforループにtqdmを使うことでプログレスバーを表示できます。

In [ ]:
from tqdm import tqdm

# トレーニング
for i_epoch in tqdm(range(num_steps)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()